In [2]:
##!pip uninstall spacy
##!pip install spacy==3.5.3
##!python -m spacy download pt_core_news_sm
from collections import Counter
from collections import defaultdict
import pandas as pd
import unicodedata
import spacy
from collections import defaultdict

In [3]:
nlp = spacy.load("pt_core_news_sm")

def lematizar(texto):
    doc = nlp(texto)
    return ''.join(token.lemma_ for token in doc)

def remover_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def temEspaco(texto):
    return ' ' in texto

In [10]:
df = pd.read_excel('rawdata.xlsx', sheet_name='frases_categorias')

palavras_genericas = ['perfume', 'fragrancia', 'cheiroso', 'cheirar', 'cheirando', 'cheiro', 'em o', 'de o', 'gostar', 'deixar','roupa','ficar','muito']
palavras_genericas.append('porque')
dados_finais = []


categorias = df['categoria'].unique()
for categoria in categorias:
    filtrado = df[df['categoria'] == categoria]
    todas_palavras = []

    for frase in filtrado['frase']:
        bruto_palavras = frase.split()
        for palavra in bruto_palavras:
            palavra_proc = lematizar(remover_acentos(palavra.lower()))
            if len(palavra_proc) > 3 and palavra_proc not in palavras_genericas and not temEspaco(palavra_proc):
                todas_palavras.append(palavra_proc)

    contagem = Counter(todas_palavras)
    top_10 = contagem.most_common(10)

    for palavra, freq in top_10:
        dados_finais.append({'categoria': categoria, 'palavra': palavra, 'quantasvezes_aparece': freq})


df_resultado = pd.DataFrame(dados_finais)
df_resultado.to_csv('resultado.csv', index=False, encoding='utf-8-sig')


In [11]:
def classificar_frase(frase, df_resultado):
    palavras_frase = frase.split()
    palavras_processadas = [
        lematizar(remover_acentos(p.lower()))
        for p in palavras_frase
        if len(p) > 3 and not temEspaco(p)
    ]

    categorias_palavras_peso = defaultdict(dict)
    for _, row in df_resultado.iterrows():
        palavra = row['palavra']
        categoria = row['categoria']
        peso = row['quantasvezes_aparece']
        categorias_palavras_peso[categoria][palavra] = peso


    contagem_ponderada = {}
    for categoria, palavras_pesos in categorias_palavras_peso.items():
        peso_total = sum(
            palavras_pesos.get(p, 0) for p in palavras_processadas
        )
        contagem_ponderada[categoria] = peso_total

    top_categorias = sorted(
        contagem_ponderada.items(),
        key=lambda x: x[1],
        reverse=True
    )[:3]

    return top_categorias

In [ ]:
df_teste = pd.read_excel('frases_teste.xlsx', sheet_name='frases')
lista_test = df_teste['frase'].tolist()
for frase in lista_test:
    print(f'Frase: {frase}')
    resultado = classificar_frase(frase, df_resultado)
    print(f'Categorias: {resultado}\n')


Frase: Cheiro doce
Categorias: [('Perfume Characteristic', 12), ('No dislikes', 1), ('Pleasant perfume', 0)]

Frase: Cheiro de ervas
Categorias: [('Perfume Characteristic', 0), ('Pleasant perfume', 0), ('No dislikes', 0)]

Frase: Refrescante, limpeza, suave
Categorias: [('Perfume intensity ', 79), ('Pleasant perfume', 24), ('Perfume Characteristic', 16)]

Frase: É refrescante, suave e cremoso
Categorias: [('Perfume intensity ', 79), ('Pleasant perfume', 24), ('Perfume Characteristic', 16)]

Frase: Refrescancia
Categorias: [('Perfume Characteristic', 0), ('Pleasant perfume', 0), ('No dislikes', 0)]

Frase: Fragrância
Categorias: [('Perfume Characteristic', 0), ('Pleasant perfume', 0), ('No dislikes', 0)]

Frase: Que o cheiro não é enjoativo
Categorias: [('Pleasant perfume', 9), ('No dislikes', 4), ('Perfume Characteristic', 0)]

Frase: O cheiro que ele deixa no cabelo
Categorias: [('Perfume intensity ', 18), ('Perfume Characteristic', 16), ('Perfume long lasting', 8)]

Frase: O cheirinh

In [7]:
## Exemplo de Uso da SpaCy para Lematização

texto = "Fizemos muitas casinhas coloridas"
doc = nlp(texto)

for token in doc:
    print(f"Token: {token.text} | Lema: {token.lemma_} | POS: {token.pos_}")

Token: Fizemos | Lema: Fizemos | POS: VERB
Token: muitas | Lema: muito | POS: DET
Token: casinhas | Lema: casinha | POS: NOUN
Token: coloridas | Lema: colorer | POS: VERB
